In [1]:
import requests
from bs4 import BeautifulSoup
from requests.packages.urllib3.exceptions import InsecureRequestWarning
import re
import nltk
import ssl

In [2]:
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

from nltk.corpus import stopwords

stop = stopwords.words('english')

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)





In [75]:
def extract_phone_numbers(string):
    r = re.compile(r'(\d{4}[-\.\s]??\d{4}[-\.\s]??\d{4}|\(\d{}\)\s*\d{4}[-\.\s]??\d{4}|\d{4}[-\.\s]??\d{4})')
    phone_numbers = r.findall(string)
    return phone_numbers

def extract_email_addresses(string):
    string = string.replace("[at]","@")
    r = re.compile(r'[\w\.-]+@[\w\.-]+')
    return r.findall(string)


def ie_preprocess(document):
    document = ' '.join([i for i in document.split() if i not in stop])
    sentences = nltk.sent_tokenize(document)
    sentences = [nltk.word_tokenize(sent) for sent in sentences]
    sentences = [nltk.pos_tag(sent) for sent in sentences]
    return sentences

def extract_names(document):
    names = []
    sentences = ie_preprocess(document)
    for tagged_sentence in sentences:
        for chunk in nltk.ne_chunk(tagged_sentence):
            if type(chunk) == nltk.tree.Tree:
                if chunk.label() == 'PERSON':
                    names.append(' '.join([c for c in chunk]))
    return names, extract_email_addresses(document), extract_phone_numbers(document)




In [76]:
url = "https://www.iitb.ac.in/en/education/academic-divisions"

r = requests.get(url, verify=False)
htmlContent = r.content
soup = BeautifulSoup(htmlContent, "html.parser")


In [84]:
departments = ['Biosciences and Bioengineering']
print("\n")
for dept_name in departments:
    print(dept_name+"-------------------------------+=+++++++++++++")
    try:
        for list in soup.findAll('a'):
            if list.text.strip() == dept_name:
                d = str(list.get('href'))
                dept = requests.get(d,verify=False)
                deptContent = dept.content
                soup_dept = BeautifulSoup(deptContent,"html.parser")

        for list in soup_dept.findAll('a'):
            if list.text.strip() == "Faculty":
                f = list.get('href')
                try:
                    faculty = requests.get(f,verify=False)
                except:
                    print(d+f)
                    faculty = requests.get(d+f, verify=False)
                facultyContent = faculty.content
                soup_faculty = BeautifulSoup(facultyContent, "html.parser")

        table_body = soup_faculty.find('tbody')
        data=[]
        rows = table_body.find_all('tr')
        for row in rows:
            cols = row.find_all('td')
            cols = [ele.text.strip() for ele in cols]
            data.append([ele for ele in cols if ele])

        for i in data:
            for j in i:
                print(j) 
    except:
        names = extract_names(str(soup_faculty.text.strip()))
        print(names)
            



Biosciences and Bioengineering-------------------------------+=+++++++++++++
(['Faculty', 'Facebook', 'Twitter Biosciences BioengineeringIndian Institute Technology Bombay Home', 'Areas Facilities', 'Bombay Toggle Menu Faculty Members Balaji', 'Kanekar', 'Kumar', 'Patankar', 'Patkar', 'Paul', 'Punekar', 'Adjunct Faculty Soni', 'Faculty Banerjee', 'Us Department Biosciences BioengineeringIndian Institute Technology BombayPowai', 'Facebook Twitter', 'Seminars Seminar Room Booking Useful Forms', 'Moodle Institute Calendar Quick', 'Calendar Commemorating Prof Rinti Banerjee', 'Bombay Scroll', 'Areas Facilities'], ['balaji@iitb.ac.inPhone', 'abanerjee@iitb.ac.inPhone', 'pbhaumik@iitb.ac.inPhone', 'santanughosh@iitb.ac.inPhone', 'sandipk@iitb.ac.inPhone', 'nkanekar@iitb.ac.inPhone', 'kirankondabagil@iitb.ac.inPhone', 'ashutoshk@iitb.ac.inPhone', 'akunwar@iitb.ac.inPhone', 'samirmaji@iitb.ac.inPhone', 'rmallik@iitb.ac.inPhone', 'rmanch@iitb.ac.inPhone', 'mukherji@iitb.ac.inPhone', 'sreelaja